In [128]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 500
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import LogNorm
from matplotlib.patches import FancyArrowPatch
from astropy.visualization import SqrtStretch, LinearStretch, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import pandas as pd
import re
from astropy.io import fits
import os
from scipy.ndimage import map_coordinates
from scipy.stats import binned_statistic
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

from matplotlib.lines import Line2D
os.chdir("/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling")

In [129]:
# Model Opening
model_path = 'Sersic/SIE+SHEAR'
model_ver = 'SIE_POS_SHEAR'
lens_name = f'"{model_path}"'

if 'POS+FLUX' in model_ver:
    constraint = 'pos_flux'
elif 'POS' in model_ver:
    constraint = 'pos'


# Load the data
with open(model_path + '/' + model_ver + '_optresult' + '.dat', 'r') as file:
    opt_result = file.readlines()

opt_result

['------------------------------------------\n',
 'optimize ndim=17\n',
 'run 1: 657 lens models calculated\n',
 'chi^2 = 2.667117e-06  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 2.667117e-06 2.667117e-06 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.577517e+02  2.081264e+01  2.075977e+01  1.379965e-01  1.482400e+01  0.000000e+00  0.000000e+00 \n',
 'lens   sers    0.2613  7.822934e+10  2.081264e+01  2.075977e+01  1.917400e-01  9.715570e+01  5.188100e+01  1.442400e+00 \n',
 'lens   sers    0.2613  1.026991e+10  2.081264e+01  2.075977e+01  5.818500e-01  8.090880e+01  5.979260e+00  5.737000e-01 \n',
 'lens   pert    0.2613  1.000000e+00  2.088401e+01  2.065055e+01  3.150772e-02  1.352195e+02  0.000000e+00  0.000000e+00 \n',
 'point  1.0000  2.081896e+01  2.0777

In [130]:
# Find the last line with 'optimize' in it
last_optimize_index = None
for idx in range(len(opt_result) - 1, -1, -1):
    if 'optimize' in opt_result[idx]:
        last_optimize_index = idx
        last_optimize_line = opt_result[idx]
        break
if last_optimize_index is None:
    raise ValueError("No line with 'optimize' found in the file.")

print(f"Last 'optimize' line found at index: {last_optimize_index}")

# Extract everything after the last 'optimize' line
opt_result = opt_result[last_optimize_index + 1:]

opt_result


Last 'optimize' line found at index: 18


['run 2: 87 lens models calculated\n',
 'chi^2 = 2.667117e-06  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 2.667117e-06 2.667117e-06 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.577517e+02  2.081264e+01  2.075977e+01  1.379965e-01  1.482400e+01  0.000000e+00  0.000000e+00 \n',
 'lens   sers    0.2613  7.822934e+10  2.081264e+01  2.075977e+01  1.917400e-01  9.715570e+01  5.188100e+01  1.442400e+00 \n',
 'lens   sers    0.2613  1.026991e+10  2.081264e+01  2.075977e+01  5.818500e-01  8.090880e+01  5.979260e+00  5.737000e-01 \n',
 'lens   pert    0.2613  1.000000e+00  2.088501e+01  2.065055e+01  3.150772e-02  1.352195e+02  0.000000e+00  0.000000e+00 \n',
 'point  1.0000  2.081896e+01  2.077744e+01 \n',
 '------------------------------------------\n']

In [131]:
# Count the number of lines that start with 'lens'
lens_count = sum(1 for line in opt_result if line.startswith('lens'))
print(f"Number of 'lens' lines: {lens_count}")

# Initialize a dictionary to hold the lens parameters
lens_params_dict = {}

# Extract the lens parameters
lens_params = []
for line in opt_result:
    if line.startswith('lens'):
        # Extract the line and split it into parts
        parts = re.split(r'\s+', line.strip())
        print(parts)
        # Extract the lens name and parameters
        lens_name = parts[1]
        params = [float(x) for x in parts[2:]]

        # Store the parameters in the dictionary, allowing duplicate names as separate entries
        key = lens_name
        count = 1
        while key in lens_params_dict:
            key = f"{lens_name}_{count}"
            count += 1
        lens_params_dict[key] = params
        lens_params.append((key, params))

# Remove the first lens parameter
if lens_params:
    for i in range(len(lens_params)):
        lens_name, params = lens_params[i]
        lens_params_dict[lens_name] = params[1:]


# Number of len profiles
num_lens_profiles = len(lens_params_dict)
print(f"Number of lens profiles: {num_lens_profiles}")
    
lens_params_dict

Number of 'lens' lines: 4
['lens', 'sie', '0.2613', '1.577517e+02', '2.081264e+01', '2.075977e+01', '1.379965e-01', '1.482400e+01', '0.000000e+00', '0.000000e+00']
['lens', 'sers', '0.2613', '7.822934e+10', '2.081264e+01', '2.075977e+01', '1.917400e-01', '9.715570e+01', '5.188100e+01', '1.442400e+00']
['lens', 'sers', '0.2613', '1.026991e+10', '2.081264e+01', '2.075977e+01', '5.818500e-01', '8.090880e+01', '5.979260e+00', '5.737000e-01']
['lens', 'pert', '0.2613', '1.000000e+00', '2.088501e+01', '2.065055e+01', '3.150772e-02', '1.352195e+02', '0.000000e+00', '0.000000e+00']
Number of lens profiles: 4


{'sie': [157.7517, 20.81264, 20.75977, 0.1379965, 14.824, 0.0, 0.0],
 'sers': [78229340000.0, 20.81264, 20.75977, 0.19174, 97.1557, 51.881, 1.4424],
 'sers_1': [10269910000.0,
  20.81264,
  20.75977,
  0.58185,
  80.9088,
  5.97926,
  0.5737],
 'pert': [1.0, 20.88501, 20.65055, 0.03150772, 135.2195, 0.0, 0.0]}

In [132]:
# Initialize a dictionary to hold the lens parameters
set_point_dict = {}

# Extract the lens parameters
set_points = []
for line in opt_result:
    if line.startswith('point'):
        # Extract the line and split it into parts
        parts = re.split(r'\s+', line.strip())
        print(parts)

set_point_values = [float(x) for x in parts[1:]]

set_point_values

['point', '1.0000', '2.081896e+01', '2.077744e+01']


[1.0, 20.81896, 20.77744]

In [133]:
# Use generic column names: param1, param2, ...
df = pd.DataFrame()
rows = []
max_param_len = 0

for lens_name, params in lens_params_dict.items():
    row = {'Lens Name': lens_name}
    for i, val in enumerate(params):
        row[f'param{i+1}'] = val
    rows.append(row)
    if len(params) > max_param_len:
        max_param_len = len(params)

columns = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len)]
df = pd.DataFrame(rows, columns=columns)

# df['Lens Name'] = df['Lens Name'].apply(lambda x: f'"{x}"')

df

,Lens Name,param1,param2,param3,param4,param5,param6,param7
0,sie,1.577517e+02,20.81264,20.75977,0.137996,14.8240,0.00000,0.0000
1,sers,7.822934e+10,20.81264,20.75977,0.191740,97.1557,51.88100,1.4424
2,sers_1,1.026991e+10,20.81264,20.75977,0.581850,80.9088,5.97926,0.5737
3,pert,1.000000e+00,20.88501,20.65055,0.031508,135.2195,0.00000,0.0000


In [134]:
for i in range(len(df)):
    if df['Lens Name'][i] == 'sers_1':
        df['Lens Name'][i] = 'sers'

df

/var/folders/tm/964hg0yn70x0ddsqcj4bsf8r0000gn/T/ipykernel_81692/4180423638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lens Name'][i] = 'sers'


,Lens Name,param1,param2,param3,param4,param5,param6,param7
0,sie,1.577517e+02,20.81264,20.75977,0.137996,14.8240,0.00000,0.0000
1,sers,7.822934e+10,20.81264,20.75977,0.191740,97.1557,51.88100,1.4424
2,sers,1.026991e+10,20.81264,20.75977,0.581850,80.9088,5.97926,0.5737
3,pert,1.000000e+00,20.88501,20.65055,0.031508,135.2195,0.00000,0.0000


In [135]:
with open('write_lens' + '.py', 'r') as file:
    py = file.readlines()

py

['#!/usr/bin/env python\n',
 'import glafic\n',
 '\n',
 "glafic.init(0.3, 0.7, -1.0, 0.7, 'Sersic/SIE+SHEAR/SIE_POS_SHEAR', -0.78, -0.78, 0.78, 0.78, 0.012, 0.012, 1, verb = 0)\n",
 '\n',
 "glafic.set_secondary('chi2_splane 1', verb = 0)\n",
 "glafic.set_secondary('chi2_checknimg 0', verb = 0)\n",
 "glafic.set_secondary('chi2_restart   -1', verb = 0)\n",
 "glafic.set_secondary('chi2_usemag    1', verb = 0)\n",
 "glafic.set_secondary('hvary          0', verb = 0)\n",
 "glafic.set_secondary('ran_seed -122000', verb = 0)\n",
 '\n',
 'glafic.startup_setnum(4)\n',
 'glafic.set_lens(1, "sie", 0.261343256161012, 157.7517, 20.81264, 20.75977, 0.1379965, 14.824, 0.0, 0.0)\n',
 'glafic.set_lens(2, "sers", 0.261343256161012, 78229340000.0, 20.81264, 20.75977, 0.19174, 97.1557, 51.881, 1.4424)\n',
 'glafic.set_lens(3, "sers_1", 0.261343256161012, 10269910000.0, 20.81264, 20.75977, 0.58185, 80.9088, 5.97926, 0.5737)\n',
 'glafic.set_lens(4, "pert", 0.261343256161012, 1.0, 20.88501, 20.65055, 0.0315

In [136]:
import re

# Assume df, model_path, model_ver, and set_point_values are defined elsewhere
# Example placeholders:
# import pandas as pd
# df = pd.DataFrame({
#     'Lens Name': ['"sie"', '"sie"', '"sie"'], # Example with 3 profiles
#     'param1': [1, 1, 1], 'param2': [0, 0, 0], 'param3': [0.5, 0.5, 0.5],
#     'param4': [0, 0, 0], 'param5': [0, 0, 0], 'param6': [0, 0, 0], 'param7': [0, 0, 0]
# })
# num_lens_profiles = len(df)
# model_path = '"models"'
# model_ver = '"v1"'
# set_point_values = [0.1, 0.2, 0.3]


# Read the existing file contents
with open('write_lens.py', 'r') as f:
    lines = f.readlines()

num_lens_profiles = len(df)
lens_name = (model_path + '/' + model_ver).strip("'\"")
print(f"Processing for {num_lens_profiles} lens profiles. Final state will be determined by this number.")

new_lines = []
# --- State variables to check if the lines exist at all ---
found_set_lens_4 = False
found_setopt_4 = False
last_lens_line_index = -1

for line in lines:
    # We use 'continue' to ensure each line is handled by only one block
    # and to prevent it from falling through to the 'else' at the end.

    # --- BLOCK 1: Handles all `glafic.set_lens(4)` cases ---
    if 'glafic.set_lens(4' in line:
        found_set_lens_4 = True
        if num_lens_profiles > 3:
            print("State: 4+ lenses. Ensuring 'glafic.set_lens(4)' is active and correct.")
            idx = 3 # This is the 4th lens
            lens_row = df.iloc[idx]
            model_name = str(lens_row["Lens Name"]).strip("'\"")
            lens_str = f'glafic.set_lens({idx+1}, "{model_name}", 0.261343256161012'
            for p in range(1, 8):
                lens_str += f', {lens_row[f"param{p}"]}'
            lens_str += ')\n'
            new_lines.append(lens_str)
        else:
            print("State: <=3 lenses. Ensuring 'glafic.set_lens(4)' is a commented placeholder.")
            new_lines.append('# glafic.set_lens(4)\n')
        last_lens_line_index = len(new_lines) - 1
        continue

    # --- BLOCK 2: Handles all `glafic.setopt_lens(4)` cases ---
    elif 'glafic.setopt_lens(4' in line:
        found_setopt_4 = True
        if num_lens_profiles > 3:
            print("State: 4+ lenses. Ensuring 'glafic.setopt_lens(4)' is active.")
            new_lines.append('glafic.setopt_lens(4, 0, 0, 0, 0, 0, 0, 0, 0)\n')
        else:
            print("State: <=3 lenses. Ensuring 'glafic.setopt_lens(4)' is a commented placeholder.")
            new_lines.append('# glafic.setopt_lens(4, 0, 0, 0, 0, 0, 0, 0, 0)\n')
        last_lens_line_index = len(new_lines) - 1
        continue

    # --- BLOCK 3: Handles general `glafic.set_lens` lines (1, 2, 3) ---
    elif 'glafic.set_lens(' in line:
        m = re.search(r'glafic\.set_lens\((\d+)', line)
        if m:
            idx = int(m.group(1)) - 1
            if idx < num_lens_profiles:
                lens_row = df.iloc[idx]
                model_name = str(lens_row["Lens Name"]).strip("'\"")
                lens_str = f'glafic.set_lens({idx+1}, "{model_name}", 0.261343256161012'
                for p in range(1, 8):
                    lens_str += f', {lens_row[f"param{p}"]}'
                lens_str += ')\n'
                print(f'Found and replaced glafic.set_lens({idx+1})')
                line = lens_str
        new_lines.append(line)
        last_lens_line_index = len(new_lines) - 1
        continue
    
    # --- Other standard replacements ---
    elif 'glafic.startup_setnum(' in line:
        new_lines.append(f"glafic.startup_setnum({num_lens_profiles})\n")
    elif 'glafic.init' in line:
        new_lines.append(f"glafic.init(0.3, 0.7, -1.0, 0.7, '{lens_name}', -0.78, -0.78, 0.78, 0.78, 0.012, 0.012, 1, verb = 0)\n")
    else:
        # If none of the above match, add the line as is
        new_lines.append(line)

# --- After the loop, perform the self-healing check to add missing lines ---
if last_lens_line_index != -1:
    set_lens_4_placeholder = '# glafic.set_lens(4)\n'
    setopt_lens_4_placeholder = '# glafic.setopt_lens(4, 0, 0, 0, 0, 0, 0, 0, 0)\n'
    
    # We insert in reverse order to maintain correct positioning relative to the insertion point.
    if not found_setopt_4 and num_lens_profiles <= 3:
        print('Placeholder for "setopt_lens(4)" was missing entirely. Adding it back.')
        new_lines.insert(last_lens_line_index + 1, setopt_lens_4_placeholder)

    if not found_set_lens_4 and num_lens_profiles <= 3:
        print('Placeholder for "set_lens(4)" was missing entirely. Adding it back.')
        new_lines.insert(last_lens_line_index + 1, set_lens_4_placeholder)

# Write back once
with open('write_lens.py', 'w') as glafic_file:
    glafic_file.writelines(new_lines)

Processing for 4 lens profiles. Final state will be determined by this number.
Found and replaced glafic.set_lens(1)
Found and replaced glafic.set_lens(2)
Found and replaced glafic.set_lens(3)
State: 4+ lenses. Ensuring 'glafic.set_lens(4)' is active and correct.
State: 4+ lenses. Ensuring 'glafic.setopt_lens(4)' is active.
